## Zadania do wykonania

### Zadanie 1

Zaprojektuj klasę abstrakcyjną / rekord, która wyznacza pole, obwód oraz zwraca długości boków trójkąta (dowolnego). Klasami dziedziczącymi z tej klasy ma być trójkąt:
* równoboczny (konstruktor z jednym parametrem),
* równoramienny (konstruktor z dwoma parametrami - dwa różne boki),
* prostokątny (konstruktor z z dwoma parametrami - długości przyprostokątne).

> Przeciąż metodę `ToString()`, która ma zawierać zestaw podstawowych informacji. Zadanie można zrealizować na dwa sposoby: 
1. metody do obliczeń pola i obwodu zaimplementować w klasie abstrakcyjnej używając wzoru:

$
	P =\sqrt{p\cdot(p-A)\cdot(p-B)\cdot(p-C)}
$

gdzie: $p$ to połowa obwodu trójkąta. Symbole $A$, $B$, $C$ to długości poszczególnych boków. Oznaczyć metody obliczające pole i obwód jako abstrakcyjne i przeciążyć je w klasie szczegółowej.

Metoda `ToString` powinna być przeciążona w klasie abstrakcyjnej.

In [ ]:
public abstract class Triangle(double a, double b, double c) 
{

    private readonly double a = a;
    private readonly double b = b;
    private readonly double c = c;
    private readonly double p = (a + b + c) / 2;

    public double Area()
    {
        return Math.Sqrt(p * (p - a) * (p - b) * (p - c));
    }

    public double Perimeter()
    {
        return a + b + c;
    }

    public override string ToString()
    {
        return "a = " + a + ", b = " + b + ", c = " + c + ", p = " + p;
    }
}

// trójkąt równoboczny
public class EquilateralTriangle : Triangle
{
    private readonly double a;
    private readonly double b;
    private readonly double c;
    private readonly double p;

    public EquilateralTriangle(double a) : base(a, a, a)
    {
        this.a = a;
        this.b = a;
        this.c = a;
        this.p = (a + a + a) / 2;
    }

}

// trójkąt równoramienny
public class IsoscelesTriangle : Triangle
{
    private readonly double a;
    private readonly double b;
    private readonly double c;
    private readonly double p;

    public IsoscelesTriangle(double a, double b) : base(a, b, b)
    {
        this.a = a;
        this.b = b;
        this.c = b;
        this.p = (a + b + c) / 2;
    }

}

// trójkąt prostokątny
public class RightTriangle : Triangle
{
    private readonly double a;
    private readonly double b;
    private readonly double c;
    private readonly double p;

    public RightTriangle(double a, double b) : base(a, b, Math.Sqrt(a * a + b * b))
    {
        this.a = a;
        this.b = b;
        this.c = Math.Sqrt(a * a + b * b);
        this.p = (a + b + c) / 2;
    }

}


### Zadanie 2

Utwórz klasę generyczną kartoteka pracowników, która umożliwia:

* dodawanie/usuwanie,
* wyświetlanie,
* walidację istniejących już pracowników,
* wyszukiwanie danych. 

Klasa pracownik musi zawierać przynajmniej 5 cech. Każdy pracownik powinien posiadać stanowisko pracy. Klasa powinna zawierać walidację danych (poprzez metodę `Validate()`). Klasa pracownik powinna zawierać metody `Show()` i `IsMatch()`, z których będzie korzystała kartoteka przy wyszukiwaniu.

Przy wyświetlaniu pomocna może okazać się metoda `Format()`.


In [1]:
using System.Text.RegularExpressions;

class CatalogGeneric<T> 
{

    private List<T> catalog;

    public CatalogGeneric() {
        catalog = [];
    }

    // adding
    public void Add(T item) {
        catalog.Add(item);
    }

    // removing
    public void Remove(T item) {
        catalog.Remove(item);
    }

    // printing
    public void Print() {
        if (catalog.Count == 0) {
            Console.WriteLine("Catalog is empty");
        } else {
            foreach (T item in catalog) {
                Console.WriteLine(item.ToString());
            }
        }
    }

    // searching
    public T? Search(T catalogItem) {
        if (catalog.Count != 0 && catalog.Contains(catalogItem)) {
            return catalogItem;
        } else {
            return default(T);
        }
    }

    // validating
    public bool Validate(T item) {
        return catalog.Contains(item);
    }

}

public class Employee(string name, string surname, string position, string email)
{

    // Instance variables 
    private string name = name;
    private string surname = surname;
    private string position = position;
    private string email = email;
    private int id;


    // getters
    public String GetName() {
        return this.name;
    }

    public String GetSurname() {
        return this.surname;
    }

    public String GetPosition() {
        return this.position;
    }

    public String GetEmail() {
        return this.email;
    }

    public int GetId() {
        return this.id;
    }

    // setters
    public void SetName(String name) {
        this.name = name;
    }

    public void SetSurname(String surname) {
        this.surname = surname;
    }

    public void SetPosition(String position) {
        this.position = position;
    }

    public void SetEmail(String email) {
        this.email = email;
    }

    public void SetId(int id) {
        this.id = id;
    }

    public bool ValidateData(string name, string surname, string position, string email, int id) {
        EmployeeValidator validator = new();
        return EmployeeValidator.ValidateAll(name, surname, position, email, id);
    }

    // Show()
    public static String Show(string name, string surname, string position, string email) {
        return string.Format("Name: {0}, Surname: {1}, Position: {2}, Email: {3}", name, surname, position, email);
    }

    // IsMatch()
    public bool IsMatch(String name, String surname, String position, String email) {
        return (
            name.Equals(this.name) &&
            surname.Equals(this.surname) &&
            position.Equals(this.position) &&
            email.Equals(this.email)
            );
    }

    // ToString()
    public override string ToString() {
        return string.Format("Name: {0}, Surname: {1}, Position: {2}, Email: {3}", name, surname, position, email);
    }
}

partial class EmployeeValidator {

    const int MIN_NAME_LENGTH = 2;
    const int MAX_NAME_LENGTH = 20;
    const int MIN_POSITION_LENGTH = 2;
    const int MAX_POSITION_LENGTH = 20;
    const int MIN_ID = 0;
    const String NAME_REGEX = @"^[a-zA-Z]+$";
    const String EMAIL_REGEX = "^[A-Za-z0-9.]+@(.+)$";  // Regex pattern : any number of letters, numbers and dots, then @, 
                                                        // then any number of letters, numbers and dots and $ asserting the end of the string

    // validate name and surname
    private static bool ValidateName(String name) {
        return !String.IsNullOrEmpty(name) && name.Length > MIN_NAME_LENGTH && name.Length < MAX_NAME_LENGTH && Regex.IsMatch(name, NAME_REGEX);
    }

    // validate email address against regex
    private static bool ValidateEmail(String email) {
        return !String.IsNullOrEmpty(email) && Regex.IsMatch(email, EMAIL_REGEX);
    }

    private static bool ValidatePosition(String position) {
        return !String.IsNullOrEmpty(position) && position.Length > MIN_POSITION_LENGTH && position.Length < MAX_POSITION_LENGTH && Regex.IsMatch(position, NAME_REGEX);
    }

    private static bool ValidateId(int id) {
        return id > MIN_ID;
    }

    public static bool ValidateAll(String name, String surname, String position, String email, int id) {
        return (
            ValidateName(name) &&
            ValidateName(surname) &&
            ValidatePosition(position) &&
            ValidateEmail(email) &&
            ValidateId(id)
            );
    }

    // This does not work here
    // [GeneratedRegex(NAME_REGEX)]
    // private static partial Regex MyRegex();
    // [GeneratedRegex(EMAIL_REGEX)]
    // private static partial Regex MyRegex1();
}

Error: (169,34): error CS8795: Partial method 'EmployeeValidator.MyRegex()' must have an implementation part because it has accessibility modifiers.
(171,34): error CS8795: Partial method 'EmployeeValidator.MyRegex1()' must have an implementation part because it has accessibility modifiers.


### Zadanie 3

Dodaj do kartoteki z pierwszego zadania odczytywanie i zapisywanie danych w formacie TXT, XML oraz JSON (minimum jeden). Wykorzystaj do tego celu wzorzec projektowy budowniczy. Abstrakcję konkretnego zapisu można przekazać do konstruktora kartoteki, która posiada metodę `Save()`.


In [1]:
// kod z poprzedniego zadania

abstract class Saver {

    public abstract bool saveData(Employee employee);

}

class JSONSaver : Saver {

    @Override
    public bool saveData(Employee employee) {
        
        try {
            // create a writer
            FileWriter writer = new FileWriter("data.json");
            // create a JSON object
            JSONObject obj = new JSONObject();
            // put data to the object
            obj.put("name", this.name);
            obj.put("surname", this.surname);
            obj.put("position", this.position);
            obj.put("email", this.email);

            // write the object to the file
            writer.write(obj.toJSONString());

            // close the writer
            writer.close();

            return true;
        }

        catch (Exception e) {
            return false;
        }

    }

}


class XMLSaver : Saver {

    @Override
    public bool saveData(Employee employee) {
        
        try {


            return true;
        }

        catch (Exception e) {
            return false;
        }

    }

}

class TextSaver : Saver {

    @Override
    public bool saveData(Employee employee) {
        
        try {
            return true;
        }

        catch (Exception e) {
            return false;
        }

    }

}

class AbstractReader {

    abstract Employee readData();

}

class JSONReader : AbstractReader {

} 

class XMLReader : AbstractReader {

}

class TextReader : AbstractReader {

}

Error: (9,17): error CS1514: { expected
(9,17): error CS1513: } expected
(11,5): error CS1014: A get or set accessor expected
(11,14): error CS1513: } expected
(24,1): error CS7017: Member definition, statement, or end-of-file expected


### Zadanie 4
 
Rozszerz poprzednie zadanie o możliwość szyfrowania danych metodą cezara. Blok przesunięć powinien być większy, niż jeden. W celu jego przechowywania wykorzystaj klasę `ConfigurationManager` (najpierw należy dodać referencję do biblioteki `System.Configuration`).

In [ ]:
class CeasarCipher {

    private int shift;

    public CeasarCipher(int shift) {
        this.shift = shift;
    }

    public String encrypt(String text) {
        String encrypted = "";
        for (int i = 0; i < text.length(); i++) {
            char c = (char)(text.charAt(i) + shift);
            if (c > 'z')
                encrypted += (char)(text.charAt(i) - (26-shift));
            else
                encrypted += (char)(text.charAt(i) + shift);
        }
        return encrypted;
    }

    public String decrypt(String text) {
        String decrypted = "";
        for (int i = 0; i < text.length(); i++) {
            char c = (char)(text.charAt(i) - shift);
            if (c < 'a')
                decrypted += (char)(text.charAt(i) + (26-shift));
            else
                decrypted += (char)(text.charAt(i) - shift);
        }
        return decrypted;
    }


}